# Entrenador de modelos

Este script contiene el código para entrenar un modelo de regresión lineal a partir de datos de ventas históricos. El modelo como tal es muy sencillo y probablemente no muy bueno, pero el objetivo del ejercicio es mostrar la arquitectura del sistema completo.

Una vez se ejecuta el comando `.fit` el modelo está listo para predecir. En ese momento el modelo es un objeto de Python en memoria. ¿Qué ocurre si el proceso que lo ha entrenado se cae por un cuelgue, un fallo en el datacenter, etc.? ¿Cómo podemos usar el modelo en varios proceso para poder paralelizar las predicciones? Es habitual guardar el modelo en disco con `pickle` (aunque la librería de scikit-learn ofrece [alternativas](https://scikit-learn.org/stable/modules/model_persistence.html)). Este proceso se denomina serialización y es común a cualquier lenguaje de programación.

Aparte de la simplicidad del modelo, es relevante observar que no hay una sola referencia a Kafka o a sus topics: la entrada es el fichero `historic.csv` y la salida es un fichero `.pickle`. Este código puede ejecutarse al margen del estado de los topics.

In [1]:
import pandas as pd
import numpy as np
from ejercicios.houses import SEED, MODEL_STORE, HISTORIC
np.random.seed(SEED)

In [2]:
ds = pd.read_csv(HISTORIC)

In [3]:
features = list(ds.columns)
target = 'price'
features.remove('price')
features.remove('id')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(ds[features], ds[target], random_state=SEED, test_size=0.3)

# Creación de un modelo
model = LinearRegression()
model.fit(x_train, y_train)

predict_train = model.predict(x_train)
predict_test = model.predict(x_test)

La celda anterior pretende ser una version muy simplificada de un pipeline de machine learning. Podríamos añadir etapas como diferentes algoritmos, generación de variables, optimización de parámetros, etc., complicando el modelo tanto como necesitemos. El objetivo, en cualquier caso, es conseguir un objeto `model` que nos sirva para ejecutar un `model.predict`. Eso sí, aquí no vamos a predecir, sólo vamos a guardar el modelo.

In [4]:
# Evaluación de R2
# Lo imprimimos simplemente para comprobar que sucesivos entrenamiento tendrán diferente rendimiento
print('R^2 en entrenamiento es: ', model.score(x_train, y_train))
print('R^2 en validación es: ', model.score(x_test, y_test))

R^2 en entrenamiento es:  0.6223331320830893
R^2 en validación es:  0.6206864179192404


In [5]:
import pickle
f = open(MODEL_STORE, 'wb')
pickle.dump(model, f)
f.flush()
f.close()